# Spatial Analysis of Zomato
***

## Introduction

## Business Understanding

## Data Understanding

In [1]:
# Importing relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
import folium
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster
from folium import Marker

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
data = pd.read_csv(r'Data/zomato.csv')
type(data)

pandas.core.frame.DataFrame

above "data" belongs to dataframe data-structure

In [3]:
# Getting first 3 rows of data
data.head(3) 

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari


In [4]:
# Getting all the columns of dataframe
data.columns

Index(['url', 'address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'phone', 'location', 'rest_type', 'dish_liked', 'cuisines',
       'approx_cost(for two people)', 'reviews_list', 'menu_item',
       'listed_in(type)', 'listed_in(city)'],
      dtype='object')

In [5]:
# Getting dimensions of dataframe
data.shape

(51717, 17)

## Data Preprocessing

In [6]:
data.duplicated().sum()

0

In [7]:
data.isnull().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7775
votes                              0
phone                           1208
location                          21
rest_type                        227
dish_liked                     28078
cuisines                          45
approx_cost(for two people)      346
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [8]:
data.dropna(subset=['location'], inplace=True)
data.isnull().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7754
votes                              0
phone                           1187
location                           0
rest_type                        206
dish_liked                     28057
cuisines                          24
approx_cost(for two people)      325
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [9]:
df = data.copy()

In [10]:
df['location']

0                      Banashankari
1                      Banashankari
2                      Banashankari
3                      Banashankari
4                      Basavanagudi
                    ...            
51712                    Whitefield
51713                    Whitefield
51714                    Whitefield
51715    ITPL Main Road, Whitefield
51716    ITPL Main Road, Whitefield
Name: location, Length: 51696, dtype: object

In [11]:
df['location'] = df['location'] + ' , Bangalore , Karnataka , India'
df['location']

0             Banashankari , Bangalore , Karnataka , India
1             Banashankari , Bangalore , Karnataka , India
2             Banashankari , Bangalore , Karnataka , India
3             Banashankari , Bangalore , Karnataka , India
4             Basavanagudi , Bangalore , Karnataka , India
                               ...                        
51712           Whitefield , Bangalore , Karnataka , India
51713           Whitefield , Bangalore , Karnataka , India
51714           Whitefield , Bangalore , Karnataka , India
51715    ITPL Main Road, Whitefield , Bangalore , Karna...
51716    ITPL Main Road, Whitefield , Bangalore , Karna...
Name: location, Length: 51696, dtype: object

In [12]:
df.dtypes

url                            object
address                        object
name                           object
online_order                   object
book_table                     object
rate                           object
votes                           int64
phone                          object
location                       object
rest_type                      object
dish_liked                     object
cuisines                       object
approx_cost(for two people)    object
reviews_list                   object
menu_item                      object
listed_in(type)                object
listed_in(city)                object
dtype: object

# Extract Latitudes and Longitudes

In [13]:
# New dataframe
rest_loc = pd.DataFrame()

rest_loc['Name'] = df['location'].unique()
rest_loc

,Name
0,"Banashankari , Bangalore , Karnataka , India"
1,"Basavanagudi , Bangalore , Karnataka , India"
2,"Mysore Road , Bangalore , Karnataka , India"
3,"Jayanagar , Bangalore , Karnataka , India"
4,"Kumaraswamy Layout , Bangalore , Karnataka , I..."
...,...
88,"West Bangalore , Bangalore , Karnataka , India"
89,"Magadi Road , Bangalore , Karnataka , India"
90,"Yelahanka , Bangalore , Karnataka , India"
91,"Sahakara Nagar , Bangalore , Karnataka , India"


In [14]:
geolocator = Nominatim(user_agent='app1', timeout=None)

In [15]:
lat = []
lon = []

for name in rest_loc['Name']:
    location = geolocator.geocode(name)
    
    if location is None:
        lat.append(np.nan)
        lon.append(np.nan)
    else:
        lat.append(location.latitude)
        lon.append(location.longitude)
        
print(lat)

[12.9152186, 12.9417261, 12.9466619, 12.9292731, 12.9081487, 12.9274413, 12.9658625, 12.9055682, 12.9120761, 12.9287596, 12.965717999999999, 13.00196975, 12.9153309, 12.911275849999999, 12.9124938, 12.9089453, 12.9854892, 12.848759900000001, 12.9116225, 12.9552572, 12.920441, 12.9489339, 12.9575547, 12.9348429, 12.9408685, 12.9668257, 12.9364846, 13.0464531, 12.9327778, 12.93103185, 12.9696365, 12.98970315, 12.9572222, 12.9732913, 12.9277245, 12.9986827, 13.0227204, 12.9755264, 12.9750849, 12.9683054, 13.0328037, 12.9778793, 12.9660577, 12.986391, 12.9802639, 12.9744255, 12.987043, 12.9844724, 12.9822323, 12.988721250000001, 13.0358698, 12.9624669, 12.945245, 12.9678074, 13.0076091, 13.0027353, 12.9931876, 13.0093455, 12.9390255, 12.9756601, 12.957998, 12.9730367, 12.9578658, 12.957434549999999, 12.9864828, 12.944569, 13.007516, 12.9243692, 12.9282918, 12.9340114, 12.9211258, 12.9882338, 13.0141618, 13.022234699999998, 13.0422794, 13.0258087, 13.0221416, 13.0421236, 13.0784743, nan, 12

In [16]:
rest_loc['lat'] = lat
rest_loc['lon'] = lon

rest_loc

,Name,lat,lon
0,"Banashankari , Bangalore , Karnataka , India",12.915219,77.573621
1,"Basavanagudi , Bangalore , Karnataka , India",12.941726,77.575502
2,"Mysore Road , Bangalore , Karnataka , India",12.946662,77.530090
3,"Jayanagar , Bangalore , Karnataka , India",12.929273,77.582423
4,"Kumaraswamy Layout , Bangalore , Karnataka , I...",12.908149,77.555318
...,...,...,...
88,"West Bangalore , Bangalore , Karnataka , India",13.022235,77.567183
89,"Magadi Road , Bangalore , Karnataka , India",12.975608,77.555356
90,"Yelahanka , Bangalore , Karnataka , India",13.100698,77.596345
91,"Sahakara Nagar , Bangalore , Karnataka , India",13.062147,77.580061


In [17]:
rest_loc.isnull().sum()

Name    0
lat     2
lon     2
dtype: int64

In [18]:
rest_loc[rest_loc['lat'].isnull()]

,Name,lat,lon
79,"Rammurthy Nagar , Bangalore , Karnataka , India",NaN,NaN
85,"Sadashiv Nagar , Bangalore , Karnataka , India",NaN,NaN


Google search of coordinates for missing

In [19]:
# for Rammurthy Nagar , Bangalore 13.0163° N, 77.6785° E
rest_loc['lat'][79] = 13.0184435
rest_loc['lon'][79] = 77.6781215

rest_loc['lat'][85] = 13.010316
rest_loc['lon'][85] = 77.580569

rest_loc.isnull().sum()

Name    0
lat     0
lon     0
dtype: int64

## Restaurants Density
Where are most restaurants located in Bengalure City

In [20]:
restaurant_locations = df['location'].value_counts().reset_index()
restaurant_locations

,index,location
0,"BTM , Bangalore , Karnataka , India",5124
1,"HSR , Bangalore , Karnataka , India",2523
2,"Koramangala 5th Block , Bangalore , Karnataka ...",2504
3,"JP Nagar , Bangalore , Karnataka , India",2235
4,"Whitefield , Bangalore , Karnataka , India",2144
...,...,...
88,"West Bangalore , Bangalore , Karnataka , India",6
89,"Yelahanka , Bangalore , Karnataka , India",6
90,"Jakkur , Bangalore , Karnataka , India",3
91,"Rajarajeshwari Nagar , Bangalore , Karnataka ,...",2


In [21]:
restaurant_locations.columns = ['Name', 'Count']
restaurant_locations.head()

,Name,Count
0,"BTM , Bangalore , Karnataka , India",5124
1,"HSR , Bangalore , Karnataka , India",2523
2,"Koramangala 5th Block , Bangalore , Karnataka ...",2504
3,"JP Nagar , Bangalore , Karnataka , India",2235
4,"Whitefield , Bangalore , Karnataka , India",2144


Merge 2 dataframes to get Name, 'Count', lat' and 'lot. Have a common column 'Name'

In [22]:
beng_restaurant_locations = rest_loc.merge(restaurant_locations, on='Name')
beng_restaurant_locations

,Name,lat,lon,Count
0,"Banashankari , Bangalore , Karnataka , India",12.915219,77.573621,906
1,"Basavanagudi , Bangalore , Karnataka , India",12.941726,77.575502,684
2,"Mysore Road , Bangalore , Karnataka , India",12.946662,77.530090,22
3,"Jayanagar , Bangalore , Karnataka , India",12.929273,77.582423,1926
4,"Kumaraswamy Layout , Bangalore , Karnataka , I...",12.908149,77.555318,195
...,...,...,...,...
88,"West Bangalore , Bangalore , Karnataka , India",13.022235,77.567183,6
89,"Magadi Road , Bangalore , Karnataka , India",12.975608,77.555356,34
90,"Yelahanka , Bangalore , Karnataka , India",13.100698,77.596345,6
91,"Sahakara Nagar , Bangalore , Karnataka , India",13.062147,77.580061,53


### Heatmap

In [23]:
def generate_basemap():
    basemap = folium.Map(location=[12.97, 77.59])
    return basemap

basemap = generate_basemap()
basemap

In [24]:
HeatMap(beng_restaurant_locations[['lat', 'lon', 'Count']]).add_to(basemap)
basemap

### Marker Cluster Analysis

In [25]:
basemap2 = generate_basemap()

FastMarkerCluster(beng_restaurant_locations[['lat', 'lon', 'Count']]).add_to(basemap2)

basemap2

### Plotting all locations 

In [26]:
basemap3 = generate_basemap()

for idx, row in beng_restaurant_locations.iterrows():
    Marker(location=[row['lat'], row['lon']], popup=(row['Name'],row['Count'])).add_to(basemap3)

basemap3

### where are the restaurants with high average rating located

In [27]:
df['rate'].unique()

array(['4.1/5', '3.8/5', '3.7/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5',
       '3.9/5', '3.1/5', '3.0/5', '3.2/5', '3.3/5', '2.8/5', '4.4/5',
       '4.3/5', 'NEW', '2.9/5', '3.5/5', nan, '2.6/5', '3.8 /5', '3.4/5',
       '4.5/5', '2.5/5', '2.7/5', '4.7/5', '2.4/5', '2.2/5', '2.3/5',
       '3.4 /5', '-', '3.6 /5', '4.8/5', '3.9 /5', '4.2 /5', '4.0 /5',
       '4.1 /5', '3.7 /5', '3.1 /5', '2.9 /5', '3.3 /5', '2.8 /5',
       '3.5 /5', '2.7 /5', '2.5 /5', '3.2 /5', '2.6 /5', '4.5 /5',
       '4.3 /5', '4.4 /5', '4.9/5', '2.1/5', '2.0/5', '1.8/5', '4.6 /5',
       '4.9 /5', '3.0 /5', '4.8 /5', '2.3 /5', '4.7 /5', '2.4 /5',
       '2.1 /5', '2.2 /5', '2.0 /5', '1.8 /5'], dtype=object)

In [28]:
df['rate'].isnull().sum()/len(df)*100

14.999226245744351

In [30]:
df.dropna(subset=['rate'], inplace=True)

df['rate'].replace('NEW', '0', inplace=True)
df['rate'].replace('-', '0', inplace=True)

df['rating'] = df['rate'].str.replace('/5', '')
df['rating'] = df['rating'].astype(float)

df['rating'].unique()

array([4.1, 3.8, 3.7, 3.6, 4.6, 4. , 4.2, 3.9, 3.1, 3. , 3.2, 3.3, 2.8,
       4.4, 4.3, 0. , 2.9, 3.5, 2.6, 3.4, 4.5, 2.5, 2.7, 4.7, 2.4, 2.2,
       2.3, 4.8, 4.9, 2.1, 2. , 1.8])

In [33]:
avg_rating_df = df.groupby(by=['location'], as_index=False).agg({'rating':'mean', 'name':'size'})
avg_rating_df.columns = ['Name', 'Avg_rating', 'Count']

avg_rating_df

,Name,Avg_rating,Count
0,"BTM , Bangalore , Karnataka , India",3.296128,4261
1,"Banashankari , Bangalore , Karnataka , India",3.373292,805
2,"Banaswadi , Bangalore , Karnataka , India",3.362926,499
3,"Bannerghatta Road , Bangalore , Karnataka , India",3.271677,1324
4,"Basavanagudi , Bangalore , Karnataka , India",3.478185,628
...,...,...,...
87,"West Bangalore , Bangalore , Karnataka , India",2.020000,5
88,"Whitefield , Bangalore , Karnataka , India",3.384170,1693
89,"Wilson Garden , Bangalore , Karnataka , India",3.257635,203
90,"Yelahanka , Bangalore , Karnataka , India",3.640000,5


In [44]:
avg_rating_df = avg_rating_df[avg_rating_df['Count'] > 400]
avg_rating_df

,Name,Avg_rating,Count
0,"BTM , Bangalore , Karnataka , India",3.296128,4261
1,"Banashankari , Bangalore , Karnataka , India",3.373292,805
2,"Banaswadi , Bangalore , Karnataka , India",3.362926,499
3,"Bannerghatta Road , Bangalore , Karnataka , India",3.271677,1324
4,"Basavanagudi , Bangalore , Karnataka , India",3.478185,628
6,"Bellandur , Bangalore , Karnataka , India",3.309833,1078
8,"Brigade Road , Bangalore , Karnataka , India",3.595849,1084
9,"Brookefield , Bangalore , Karnataka , India",3.374699,581
12,"Church Street , Bangalore , Karnataka , India",3.963091,550
15,"Cunningham Road , Bangalore , Karnataka , India",3.901053,475


In [43]:
ratings_locations = avg_rating_df.merge(rest_loc, on='Name')
ratings_locations

,Name,Avg_rating,Count,lat,lon
0,"BTM , Bangalore , Karnataka , India",3.296128,4261,12.911276,77.604565
1,"Banashankari , Bangalore , Karnataka , India",3.373292,805,12.915219,77.573621
2,"Banaswadi , Bangalore , Karnataka , India",3.362926,499,13.014162,77.651854
3,"Bannerghatta Road , Bangalore , Karnataka , India",3.271677,1324,12.915331,77.600027
4,"Basavanagudi , Bangalore , Karnataka , India",3.478185,628,12.941726,77.575502
5,"Bellandur , Bangalore , Karnataka , India",3.309833,1078,12.931032,77.678247
6,"Brigade Road , Bangalore , Karnataka , India",3.595849,1084,12.975085,77.607934
7,"Brookefield , Bangalore , Karnataka , India",3.374699,581,12.957435,77.714708
8,"Church Street , Bangalore , Karnataka , India",3.963091,550,13.032804,77.671724
9,"Cunningham Road , Bangalore , Karnataka , India",3.901053,475,12.987043,77.594924


In [45]:
basemap4 = generate_basemap()

HeatMap(ratings_locations[['lat', 'lon', 'Avg_rating']]).add_to(basemap4)
basemap4

## Conclusions